In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.common.exceptions import NoSuchElementException
from datetime import datetime
from queue import Queue

import requests
from bs4 import BeautifulSoup

In [ ]:
bmc_news = {'title': [], 'date': [], 'content': []}

In [16]:

response = requests.get('https://www.baguiomidlandcourier.com.ph/bar-owners-discuss-issues-w-city-hall-authorities/')
soup = BeautifulSoup(response.content, 'html.parser')
try:
    # title = driver.find_element(By.CLASS_NAME, "entry-title").text
    # date = driver.find_element(By.CLASS_NAME, "entry-meta").text
    title = soup.find('h1', {'class': 'entry-title'}).text.strip()
    date = soup.find('span', {'class': 'item-metadata posts-date'}).text.strip()

    print(title)
    print(date)
    try:
        content = soup.find('div', {'class': 'entry-content read-details'}).text.strip()

        print(content)

    except NoSuchElementException:
        pass
    contents = driver.find_elements(By.TAG_NAME, 'p') # For p elements

    for c in contents:
            content = content + " " + c.text

except NoSuchElementException:
    print("error")
    
if len(title) == 0 or len(date) == 0  or len(content) == 0 :
    print("error")
    pass
else:
    bmc_news['title'].append(title)
    bmc_news['date'].append(date)
    bmc_news['content'].append(content)

Bar owners discuss issues w/ City Hall authorities
May 7, 2023
The creation of an adjudication board headed by the City Legal Office has been proposed to look over issues concerning bar and entertainment business owners and City Hall offices that are in charge of regulatory documents.
The conduct of meetings may also be initiated to thresh out concerns on permits/licenses or on the acquisition process or procedures.
These were among the things agreed on during a dialogue between bar and entertainment business owners and representatives from the Public Order and Safety Division (POSD), City Health Services, City Legal, City Planning, Development and Sustainability, and other concerned offices last week.
Among those discussed are the city’s smoke-free program where POSD Chief Daryl Longid said the provisions are clear and are enough as bases for confiscation or issuance of a notice of violation.
Longid said during inspection of establishments, details and evidence of smoking are scrutini

NameError: name 'driver' is not defined

In [ ]:
def scrape(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    try:
        title = driver.find_element(By.CLASS_NAME, "entry-title").text
        date = driver.find_element(By.CLASS_NAME, "entry-meta").text
        try:
            content = driver.find_element(By.CLASS_NAME, 'wp-element-caption').text # For captions
        except NoSuchElementException:
            pass
        contents = driver.find_elements(By.TAG_NAME, 'p') # For p elements

        for c in contents:
                content = content + " " + c.text

    except NoSuchElementException:
        print("error")

    if len(title) == 0 or len(date) == 0  or len(content) == 0 :
        print("error")
        pass
    else:
        bmc_news['title'].append(title)
        bmc_news['date'].append(date)
        bmc_news['content'].append(content)

In [ ]:
def scrape_article(driver):
    title = ''
    date = ''
    content =''

    try:
        title = driver.find_element(By.CLASS_NAME, "entry-title").text
        date = driver.find_element(By.CLASS_NAME, "entry-meta").text
        try:
            content = driver.find_element(By.CLASS_NAME, 'wp-element-caption').text # For captions
        except NoSuchElementException:
            pass
        contents = driver.find_elements(By.TAG_NAME, 'p') # For p elements

        for c in contents:
                content = content + " " + c.text

    except NoSuchElementException:
        print("error")

    if len(title) == 0 or len(date) == 0  or len(content) == 0 :
        print("error")
        pass
    else:
        bmc_news['title'].append(title)
        bmc_news['date'].append(date)
        bmc_news['content'].append(content)

    return bmc_news

In [ ]:
def scrape_bmc_news(date, return_dict):
    options = webdriver.ChromeOptions()

    url = 'https://www.baguiomidlandcourier.com.ph/'
    driver = webdriver.Chrome(options = options)
    driver.get(url)

    news_link = driver.find_element(By.CSS_SELECTOR, '.read-img.pos-rel.read-bg-img')
    next = news_link.find_element(By.TAG_NAME, 'a')
    newl = next.get_attribute('href')

    driver.implicitly_wait(10)
    driver.get(newl)

    news_date = driver.find_element(By.CLASS_NAME, 'entry-meta').text
    current_article_date = datetime.strptime(news_date, "%B %d, %Y").date()
    print(current_article_date)
    input_date = datetime.strptime(date, "%Y-%m-%d").date()
    print(input_date)

    while True:
        if current_article_date == input_date:
            # scrape article
            bmc_news = scrape_article(driver)
            
            # go to previous article
            while current_article_date == input_date:
                try:
                    link = driver.find_element(By.CLASS_NAME, 'nav-previous')
                    next = link.find_element(By.TAG_NAME, 'a')
                    newl = next.get_attribute('href')

                    driver.get(newl)
                    driver.implicitly_wait(10)

                    bmc_news = scrape_article(driver)

                    news_date = driver.find_element(By.CLASS_NAME, 'entry-meta').text
                    current_article_date = datetime.strptime(news_date, "%B %d, %Y").date()
                except NoSuchElementException:
                    break
            break;

            try:
                link = driver.find_element(By.CLASS_NAME, 'nav-previous')
                next = link.find_element(By.TAG_NAME, 'a')
                newl = next.get_attribute('href')

                driver.implicitly_wait(10)
                driver.get(newl)
                driver.implicitly_wait(10)

                news_date = driver.find_element(By.CLASS_NAME, 'entry-meta').text
                current_article_date = datetime.strptime(news_date, "%B %d, %Y").date()
            except NoSuchElementException:
                break

        elif current_article_date > input_date:
            # go to previous page
            try:
                link = driver.find_element(By.CLASS_NAME, 'nav-previous')
                next = link.find_element(By.TAG_NAME, 'a')
                newl = next.get_attribute('href')

                driver.get(newl)
                driver.implicitly_wait(10)

                news_date = driver.find_element(By.CLASS_NAME, 'entry-meta').text
                current_article_date = datetime.strptime(news_date, "%B %d, %Y").date()
            except NoSuchElementException:
                break

        else:
            # go to next page
            try:
                link = driver.find_element(By.CLASS_NAME, 'nav-next')
                next = link.find_element(By.TAG_NAME, 'a')
                newl = next.get_attribute('href')

                driver.get(newl)
                driver.implicitly_wait(10)

                news_date = driver.find_element(By.CLASS_NAME, 'entry-meta').text
                current_article_date = datetime.strptime(news_date, "%B %d, %Y").date()
            except NoSuchElementException:
                break
     
    driver.close()
    # return bmc_news
    return_dict["baguio_mc"] = bmc_news